<h1><b><code>SettingWithCopyWarning</code> in Pandas: Views vs Copies</b></h1>
<p>In this notebook, I will be covering an important concept that users must know once they get acquainted with Pandas: the <code>SettingWithCopyWarning</code> issue. As a beginner, I would often disregard this sign as the outcome of the code would not necessarily change (apparently). Now, there are <b>shallow copies</b> and <b>deep copies</b>, notions that are related to the aforementioned warning. I'll discuss the difference between these concepts as well as assessing the impact of their use in data analysis.</p>
<p>By the end of this notebook, I hope to have covered the following topics:</p>
<ul>
    <li>The definition of <b>views</b> and <b>copies</b> in NumPy and Pandas</li>
    <li>How to work with views and copies in these libraries</li>
    <li>Why <code>SettingWithCopyWarning</code> happens in Pandas</li>
    <li>How to avoid getting a <code>SettingWithCopyWarning</code> in Pandas</li>
</ul>

<h2><b>Table of contents:</b></h2>
<ul>
    <li>Example of a <code>SettingWithCopyWarning</code></li>
    <li>Views and Copies in NumPy and Pandas</li>
        <ul>
            <li>Understanding views and copies in NumPy</li>
            <li>Understanding views and copies in Pandas</li>
        </ul>
    <li>Indices and Slices in NumPy and Pandas</li>
        <ul>
            <li>Indexing in NumPy: copies and views</li>
            <li>Indexing in Pandas: copies and views</li>
        </ul>
<li>Use of Views and Copies in Pandas</li>
    <ul>
        <li>Chained Indexing and <code>SettingWithCopyWarning</code></li>
        <li>Impact of Data Types on Views, Copies, and the <code>SettingWithCopyWarning</code></li>
        <li>Hierarchical Indexing and <code>SettingWithCopyWarning</code></li>
    </ul>
<li>Change the Default <code>SettingWithCopyWarning</code> Behavior</li>
<li>Conclusion</li>
</ul>

<p>Let's start by importing the required modules and checking their versions. Then, we can move on and proceed with our discussion on this notebook's topic.</p>



In [4]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
np.__version__

'1.22.3'

In [3]:
pd.__version__

'1.4.2'

<h3><b>1. Example of a <code>SettingWithCopyWarning</code></b></h3>
<p>As previously mentioned, I tended to ignore this issue because it is <i>not</i> and <i>error</i>, but a <i>warning</i>. It might sound obvious now, but what Pandas is doing is <i>warning</i> you that you might get unwanted behavior in your code.</p>
<p>Let's create a Pandas DataFrame and observe this issue in practice:</p>

In [5]:
data = {"x": 2**np.arange(5),
        "y": 3**np.arange(5), 
        "z": np.array([45, 98, 24, 11, 64])
        }

index = ["a", "b", "c", "d", "e"]

df = pd.DataFrame(data=data, index=index)

In [6]:
df

,x,y,z
a,1,1,45
b,2,3,98
c,4,9,24
d,8,27,11
e,16,81,64
